In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from mongo import attraction_db
from model.google import Google
from dotenv import load_dotenv,find_dotenv
from pymongo import MongoClient
import random
import os
import json
import time
import csv
import os

In [81]:
url = 'https://www.google.com/maps/search/%E6%99%AF%E9%BB%9E'

driver = webdriver.Chrome()
driver.get(url)

In [82]:
def getPage():
    page_content = driver.page_source
    # 將 HTML 內容轉換成 BeautifulSoup 物件，html.parser 為使用的解析器
    soup = BeautifulSoup(page_content, 'html.parser')
    # 透過 select 使用 CSS 選擇器 選取我們要選的 html 內容
    return soup.select('.Nv2PK.THOPZb.CpccDe')

def scrollPage(times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        counter += 1

def getElement():  # 抓取目前的所有留言
    spots_list = []
    scrollPage(5)
    elements = getPage()

    for element in elements:
        name = element.select('.qBF1Pd.fontHeadlineSmall')[0].text
        url = element.select('.hfpxzc')[0].get('href')
        dataset = {}
        dataset['name'] = name
        dataset['google_url'] = url

        spots_list.append(dataset)
    return spots_list

In [297]:
lastSpot = ''
while 7 == 7:
    print(lastSpot)
    spots_list = getElement()
    if (spots_list[len(spots_list)-1]["name"] == lastSpot ):
        break
    lastSpot = spots_list[len(spots_list)-1]["name"]
    time.sleep(1)
    getElement()

csv_file = './data/spot_list/spots.csv'
if os.path.exists(csv_file):
    with open(csv_file, mode="r", newline="", encoding="utf-8-sig") as file:
        reader = csv.DictReader(file)
        existing_data = list(reader)
else:
    existing_data = []

for spot in spots_list:
    existing_data.append(spot)

print(len(existing_data))


with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    # 定義 CSV 欄位
    fieldnames = ["name", "google_url"]
    # 建立 CSV 寫入器
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(existing_data)

    print(f"已將資料寫入 CSV 檔案：{csv_file}")


後浦16藝文特區
金水學校展示館
清遠湖
7611
已將資料寫入 CSV 檔案：./data/spot_list/spots.csv


In [300]:
dataset = pd.read_csv('./data/spot_list/spots.csv')
dataset

,name,google_url
0,淡水老街,https://www.google.com/maps/place/%E6%B7%A1%E6...
1,重建街戀愛巷,https://www.google.com/maps/place/%E9%87%8D%E5...
2,一滴水紀念館,https://www.google.com/maps/place/%E4%B8%80%E6...
3,淡水情人塔 Lover's Tower,https://www.google.com/maps/place/%E6%B7%A1%E6...
4,情人橋,https://www.google.com/maps/place/%E6%83%85%E4...
...,...,...
7606,官澳風獅爺(雄),https://www.google.com/maps/place/%E5%AE%98%E6...
7607,黃卓彬洋樓,https://www.google.com/maps/place/%E9%BB%83%E5...
7608,夏墅風獅爺,https://www.google.com/maps/place/%E5%A4%8F%E5...
7609,建功嶼,https://www.google.com/maps/place/%E5%BB%BA%E5...


In [310]:
df_unique = dataset.drop_duplicates(subset="name", keep="first")
df_unique.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")
df_unique

,name,google_url
0,淡水老街,https://www.google.com/maps/place/%E6%B7%A1%E6...
1,重建街戀愛巷,https://www.google.com/maps/place/%E9%87%8D%E5...
2,一滴水紀念館,https://www.google.com/maps/place/%E4%B8%80%E6...
3,淡水情人塔 Lover's Tower,https://www.google.com/maps/place/%E6%B7%A1%E6...
4,情人橋,https://www.google.com/maps/place/%E6%83%85%E4...
...,...,...
7606,官澳風獅爺(雄),https://www.google.com/maps/place/%E5%AE%98%E6...
7607,黃卓彬洋樓,https://www.google.com/maps/place/%E9%BB%83%E5...
7608,夏墅風獅爺,https://www.google.com/maps/place/%E5%A4%8F%E5...
7609,建功嶼,https://www.google.com/maps/place/%E5%BB%BA%E5...


In [312]:
existing_names = attraction_db.attractionInfo.distinct("name")
df_filtered = df_unique[~df_unique["name"].isin(existing_names)]
df_filtered.to_csv('./data/spot_list/spotsList.csv', index=False, encoding="utf-8-sig")

In [2]:
# ============================================
# TEST
# ============================================

spotsList = pd.read_csv('./data/spot_list/spotsList.csv') 

for index, row in spotsList[105:120].iterrows():
    print(index,row['name'],row['google_url'])

105 雙溪老街 https://www.google.com/maps/place/%E9%9B%99%E6%BA%AA%E8%80%81%E8%A1%97/data=!4m7!3m6!1s0x345d5baed442085f:0x48bfb2cf00eb672f!8m2!3d25.0344966!4d121.864534!16s%2Fg%2F11gdh0wj13!19sChIJXwhC1K5bXTQRL2frAM-yv0g?authuser=0&hl=zh-TW&rclk=1
106 雙溪荷花園 https://www.google.com/maps/place/%E9%9B%99%E6%BA%AA%E8%8D%B7%E8%8A%B1%E5%9C%92/data=!4m7!3m6!1s0x345d5a1289a8d727:0x7438fc890d6e5f0a!8m2!3d25.0262271!4d121.8230223!16s%2Fg%2F11btxj5fwd!19sChIJJ9eoiRJaXTQRCl9uDYn8OHQ?authuser=0&hl=zh-TW&rclk=1
107 雙溪清水坑 https://www.google.com/maps/place/%E9%9B%99%E6%BA%AA%E6%B8%85%E6%B0%B4%E5%9D%91/data=!4m7!3m6!1s0x345d5b167c499697:0x7ce9b46a5916830c!8m2!3d25.0222089!4d121.8223017!16s%2Fg%2F11v0_6_c3g!19sChIJl5ZJfBZbXTQRDIMWWWq06Xw?authuser=0&hl=zh-TW&rclk=1
108 牡丹老街 https://www.google.com/maps/place/%E7%89%A1%E4%B8%B9%E8%80%81%E8%A1%97/data=!4m7!3m6!1s0x345d5ae3f8d714d5:0x54c177443c400bfb!8m2!3d25.0588124!4d121.8507299!16s%2Fg%2F11gdh1fy80!19sChIJ1RTX-ONaXTQR-wtAPER3wVQ?authuser=0&hl=zh-TW&rclk=1
109 三

In [3]:
# 取得 Google URL

name_to_url = {}
with open('./data/spot_list/spotsList.csv', 'r', encoding='utf-8') as spots_file:
    spots_reader = csv.reader(spots_file)
    next(spots_reader)  # 跳过标题行
    for row in spots_reader:
        name = row[0]  # 假设名称在第一列
        url = row[1]   # 假设URL在第二列
        name_to_url[name] = url

def get_url_for_name(name):
    return name_to_url.get(name, "URL not found")

In [3]:
# ============================================
# 個別處理
# ============================================

name = '櫻花大道'
url  = get_url_for_name(name)

In [24]:
data = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
tag_value = data['words'].tolist()

place_detail_data = Google().get_place_info(name,url,tag_value)
place_detail_data = json.loads(place_detail_data)
print(place_detail_data)

existing_data = attraction_db.attractionInfo.find_one({
    "$or": [
        {"place_id": place_detail_data["place_id"]},
        {"id": place_detail_data["id"]}
    ]
})
print(existing_data)
if existing_data is None:
    attraction_db.attractionInfo.insert_one(place_detail_data)
    print(f"{place_detail_data['name']}：已新增")

{'name': '櫻花大道', 'google_url': 'https://www.google.com/maps/place/%E6%AB%BB%E8%8A%B1%E5%A4%A7%E9%81%93/data=!4m7!3m6!1s0x3442ae39be7123e3:0x76aeeeafb2dd3e9e!8m2!3d25.1504227!4d121.5044983!16s%2Fg%2F11b720r0xy!19sChIJ4yNxvjmuQjQRnj7dsq_urnY?authuser=0&hl=zh-TW&rclk=1', 'address': '251新北市淡水區', 'id': 'TPH_271', 'region': '新北市', 'place_id': 'ChIJAQAAABCwQjQRLbiMeGSYq48', 'google_address': '251, Taiwan, New Taipei City, Tamsui District', 'rating': 4.1, 'user_ratings_total': 602, 'lat': 25.1687611, 'lng': 121.4614806, 'photo_reference': 'AUacShhuBu-96N4eyT7AUvHd0UnpMeSsenDHbGURoDY31itmMIPoJtP9FmzE9W3pJ8qJ2EGeXF1Hsfiz2iQUHKNhHcrv9lE3ZJPAzm-p4FueibFIVjiX_xBFURJ9A_HKHZ3zCjTDq6X6HdFvk2gNGlQnbM73skls_aG7BE9udbXaucOtX-NV', 'photo_url': 'https://lh3.googleusercontent.com/places/ANJU3DsqA6Zsa-w_iOVmBmTRj4mqfePGokgc8kUzGl_k_47ZMRqyJ9K6AJk6CN5oUOtp7Z48M9uHic_l84bYzkeFZFJLNW7n9yynNrg=s1600-w1200-h1200', 'tag': ['櫻花', '櫻花季', 'ㄧ些', '健走', '登山', '北投區', '復興三路', '櫻花樹']}
{'_id': ObjectId('64eb5a10bcda7bf0f797

In [12]:
# 檢查未成功插入資料庫的景點

existing_names = set(doc['name'] for doc in attraction_db.attractionInfo.find({}, {"name": 1}))

csv_folder_path = './data/tfidf_csv/' 
csv_files = os.listdir(csv_folder_path)
new_csv_files = [csv_file.split('.')[0] for csv_file in csv_files if csv_file.split('.')[0] not in existing_names]
print(len(new_csv_files))
print(new_csv_files)

0
[]


In [11]:
# 刪除不必要的景點
for spot in new_csv_files:
    name = spot
    csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)
        print(f"{name}.csv 已删除")

國家級照相館淡金路分館.csv 已删除
山崎尾.csv 已删除
山崗石材.csv 已删除
康荷賞花園.csv 已删除
懷恩公園牌樓.csv 已删除
成德公園.csv 已删除
本山地質公園.csv 已删除
柳杉造林地.csv 已删除
淡水區樹林口溪十一橋.csv 已删除
淡水新埔橋.csv 已删除
過港隧道.csv 已删除
酋長岩.csv 已删除
金龍湖.csv 已删除
阿桂俱樂部.csv 已删除
鼻頭角公園.csv 已删除


In [5]:
# 新增缺少的景點
for spot in new_csv_files:
    name = spot
    url = get_url_for_name(name)
    try:

        tfidf = pd.read_csv('./data/tfidf_csv/{}.csv'.format(name))
        tag_value = tfidf['words'].tolist()
        
        place_detail_data = Google().get_place_info(name,url,tag_value)
        place_detail_data = json.loads(place_detail_data)
        existing_data = attraction_db.attractionInfo.find_one({
            "$or": [
                {"place_id": place_detail_data["place_id"]},
                {"id": place_detail_data["id"]}
            ]
        })
        if existing_data is None:
            attraction_db.attractionInfo.insert_one(place_detail_data)
            print(f"{place_detail_data['name']}：已新增")
        else:
            print(f"{place_detail_data['name']}：未新增")
            csv_file_path = './data/tfidf_csv/{}.csv'.format(name)
            if os.path.exists(csv_file_path):
                os.remove(csv_file_path)
                print(f"{place_detail_data['name']}.csv 已删除")
    except Exception as e:
        print(f"錯誤訊息：{e}")
        continue

八里河岸觀景平台：已新增
和平島地質公園 Heping Island GeoPark：未新增
和平島地質公園 Heping Island GeoPark.csv 已删除
員山子分洪道：已新增
基山街觀景臺：已新增
關渡自然公園 GUANDU NATURE PARK：未新增
關渡自然公園 GUANDU NATURE PARK.csv 已删除


In [8]:
# 隨機排序景點

list = pd.read_csv('./data/spot_list/spotsList.csv')
print(list)

random.seed(42)
random_spotsList = list.sample(frac=1,random_state=random.randint(1,10000))
print(random_spotsList)

random_spotsList.to_csv('./data/spot_list/random_spotsList.csv',encoding="utf-8-sig",index=False)


        name                                         google_url
0      嶺腳觀景台  https://www.google.com/maps/place/%E5%B6%BA%E8...
1       嶺腳瀑布  https://www.google.com/maps/place/%E5%B6%BA%E8...
2       平溪瀑布  https://www.google.com/maps/place/%E5%B9%B3%E6...
3       平溪老街  https://www.google.com/maps/place/%E5%B9%B3%E6...
4       菁桐老街  https://www.google.com/maps/place/%E8%8F%81%E6...
...      ...                                                ...
2396  八二三戰史館  https://www.google.com/maps/place/%E5%85%AB%E4...
2397   田浦風獅爺  https://www.google.com/maps/place/%E7%94%B0%E6...
2398   黃卓彬洋樓  https://www.google.com/maps/place/%E9%BB%83%E5...
2399     建功嶼  https://www.google.com/maps/chplace/%E5%BB%BA%...
2400     清遠湖  https://www.google.com/maps/place/%E6%B8%85%E9...

[2401 rows x 2 columns]
               name                                         google_url
2264     將軍區長榮里木棉花道  https://www.google.com/maps/place/%E5%B0%87%E8...
1536           精英溫泉  https://www.google.com/maps/place/%E7%B2%BE%